In [3]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14


import scipy.linalg as sl
import scipy.sparse as sp
import scipy.sparse.linalg as spl

### Implement Newton's and Inexact Newton's method

In [4]:
from scipy.sparse.linalg import spsolve, gmres, spilu
from scipy.sparse.linalg import LinearOperator

def newton(F,J,x0, maxiter = 100, atol = 1.e-10, rtol = 1.e-10):
    """
    Newton's method for solving F(x) = 0
    
    """
    
    Fx0 = F(x0)
    
    reshist = []
    
    iterc = 0
    xkm1 = np.copy(x0)
    while True:
        iterc += 1
        
        Fkm1 = F(xkm1)
        Jkm1 = J(xkm1)
        
        resk = np.linalg.norm(Fkm1)/np.linalg.norm(Fx0)
        reshist.append(resk)
        print('The relative residual at step %d is %g' % (iterc, resk))
    
        #Factorize and solve
        pk  = spsolve(Jkm1, -Fkm1)
        
        #Update the solution
        xk = xkm1 + pk
        
        if (np.linalg.norm(F(xk)) <= atol + rtol*np.linalg.norm(Fx0)) or (iterc > maxiter): 
            break
        
        #Update for the next iteration
        xkm1 = xk
        
        
    return xk, iterc, reshist 



def newtongmres(F,J,x0, maxiter = 100, atol = 1.e-10, rtol = 1.e-10):
    """
    Newton's method for solving F(x) = 0
    
    """
    
    Fx0 = F(x0)
    
    reshist = []
    
    iterc = 0
    xkm1 = np.copy(x0)
    while True:
        iterc += 1
        
        Fkm1 = F(xkm1)
        Jkm1 = J(xkm1)
        
        resk = np.linalg.norm(Fkm1)/np.linalg.norm(Fx0)
        
        reshist.append(resk)
        print('The relative residual at step %d is %g' % (iterc, resk))
    
        #Factorize and solve
        prec = spilu(Jkm1)
        M = LinearOperator((n**2,n**2), lambda x: prec.solve(x))
        relres = 1.e-6*np.linalg.norm(Fkm1)
        print(np.linalg.norm(relres))
        pk, info  = gmres(Jkm1, -Fkm1, tol = relres, restart = 30)
        
        
        #Update the solution
        xk = xkm1 + pk
        
        if (np.linalg.norm(F(xk)) <= atol + rtol*np.linalg.norm(Fx0)) or (iterc > maxiter): 
            break
        
        #Update for the next iteration
        xkm1 = xk
        
        
    return xk, iterc, reshist 
    

### 2D Nonlinear Example


Consider the nonlinear PDE in two spatial dimensions
$$ -\kappa \Delta u + cu^3 = f, $$
with zero Dirichlet boundary conditions. Here $\Delta$ is the Laplacian operator and 

$$ f(x_1,x_2) = -2\pi^2 \left(\cos(2\pi x_1)\sin^2(2\pi x_2) + \sin^2(2\pi x_1)\cos(2\pi x_2) \right). $$


### Nonlinear equation and Jacobian
After discretization, the nonlinear equation becomes
$$ F(u) = \kappa A u + cu^3 - b, $$
where $A$ represents the discretized Laplacian operator and $b$ represents the discretization of $f$. 


The Jacobian operator is 
$$ J(u) = \kappa  A + 3c\text{diag}(u^2) \in \mathbb{R}^{n\times n}.$$

#### Construct the operator

In [7]:
kappa = 0.1
c = 100


n = 2**8
h = 1/(n+1)
e = np.ones((n,))
A = sp.spdiags([-e,2*e,-e], [-1,0,1], n, n, format = 'csc')
B = (kappa/h**2.)*sp.kronsum(A, A)

x = np.arange(1,n+1)*h
X,Y = np.meshgrid(x,x)

Fh = -(2*np.pi**2)*(np.cos(2*np.pi*X)*np.sin(np.pi*Y)**2. + np.sin(np.pi*X)**2.*np.cos(2*np.pi*Y))
b  = Fh.reshape((-1,))


F2d = lambda u: (B@u) + c*u**3. - b
J2d = lambda u: B + sp.spdiags([3*c*u**2.], [0], n**2, n**2, format = 'csc')

In [8]:
u0 = np.zeros((n**2,))
us, iterc, reshist = newtongmres(F2d,J2d,u0, maxiter = 100)

The relative residual at step 1 is 1
0.003576652582824119
The relative residual at step 2 is 1594.85
5.70421758679535
The relative residual at step 3 is 571.582
2.0443501238288437
The relative residual at step 4 is 189.5
0.6777750648948712
The relative residual at step 5 is 60.5235
0.21647171070970278
The relative residual at step 6 is 18.8923
0.06757107786701433
The relative residual at step 7 is 5.61172
0.020071182693613906
The relative residual at step 8 is 1.59991
0.005722323638507073
The relative residual at step 9 is 0.512902
0.0018344731150914868
The relative residual at step 10 is 0.228365
0.0008167828359635981
The relative residual at step 11 is 0.0318312
0.00011384899751062357
The relative residual at step 12 is 0.000746781
2.6709753699708305e-06
The relative residual at step 13 is 5.94989e-07
2.128070306201985e-09


In [ ]:
#### Plot the relative residual

plt.figure()
plt.semilogy(range(1,len(reshist)+1), reshist, 'k-')
plt.xlabel('Iterations', fontsize = 16)
plt.ylabel('$\|F(x_k)\|_2/ \|F(x_0)\|_2$', fontsize = 16)
plt.title('Relative Residual History', fontsize = 18)

In [ ]:

plt.pcolormesh(X,Y,us.reshape((n,n)))
plt.colorbar()
plt.title('Reconstructed Solution', fontsize = 16)